# Data Preparation for True Relationship Prediction

After predicting the co-occurence of candidates on the sentence level, the next step is to predict whether a candidate is a true relationship or just occured by chance. Through out this notebook the main events involved here are calculating summary statistics and obtaining the LSTM marginal probabilities.

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from collections import defaultdict
import csv
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import seaborn as sns
import scipy
from scipy.stats import fisher_exact
from scipy.special import logit

from sqlalchemy import and_

import tqdm

In [2]:
#Set up the environment
username = "danich1"
password = "snorkel"
dbname = "pubmeddb"

#Path subject to change for different os
database_str = "postgresql+psycopg2://{}:{}@/{}?host=/var/run/postgresql".format(username, password, dbname)
os.environ['SNORKELDB'] = database_str

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.models import Candidate, candidate_subclass

In [4]:
DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])

# Count the Number of Sentences for Each Candidate

For this block of code we are cycling through each disease-gene candidate in the database and counting the number of unique sentences and unique abstracts containing the specific candidate. **NOTE**: This section will quite a few hours to cycle through the entire database.

In [ ]:
%%time
pair_to_pmids = defaultdict(set)
pair_to_sentences = defaultdict(set)
offset = 0
chunk_size = 1e5

while True:
    cands = session.query(DiseaseGene).limit(chunk_size).offset(offset).all()
    
    if not cands:
        break
        
    for candidate in cands:
        pair = candidate.Disease_cid, candidate.Gene_cid
        pair_to_sentences[pair].add(candidate[0].get_parent().id)
        pair_to_pmids[pair].add(candidate[0].get_parent().document_id)

    offset+= chunk_size

In [ ]:
candidate_df = pd.DataFrame(
    map(lambda x: [x[0], x[1], len(pair_to_sentences[x]), len(pair_to_pmids[x])], pair_to_sentences),
    columns=["disease_id", "gene_id", "sentence_count", "abstract_count"]
    )

# Perform Fisher Exact Test on each Co-Occurence

Here we want to perform the fisher exact test for each disease-gene co-occurence. A more detailed explanation [here](https://github.com/greenelab/snorkeling/issues/26).

In [ ]:
def diffprop(obs):
    """
    `obs` must be a 2x2 numpy array.

    Returns:
    delta
        The difference in proportions
    ci
        The Wald 95% confidence interval for delta
    corrected_ci
        Yates continuity correction for the 95% confidence interval of delta.
    """
    n1, n2 = obs.sum(axis=1)
    prop1 = obs[0,0] / n1.astype(np.float64)
    prop2 = obs[1,0] / n2.astype(np.float64)
    delta = prop1 - prop2

    # Wald 95% confidence interval for delta
    se = np.sqrt(prop1*(1 - prop1)/n1 + prop2*(1 - prop2)/n2)
    ci = (delta - 1.96*se, delta + 1.96*se)

    # Yates continuity correction for confidence interval of delta
    correction = 0.5*(1/n1 + 1/n2)
    corrected_ci = (ci[0] - correction, ci[1] + correction)

    return delta, ci, corrected_ci

In [ ]:
total = candidate_df["sentence_count"].sum()
odds = []
p_val = []
expected = []
lower_ci = []
epsilon = 1e-36

for disease, gene in tqdm.tqdm(zip(candidate_df["disease_id"],candidate_df["gene_id"])):
    cond_df = candidate_df.query("disease_id == @disease & gene_id == @gene")
    a = cond_df["sentence_count"].values[0] + 1
                                        
    cond_df = candidate_df.query("disease_id != @disease & gene_id == @gene")
    b = cond_df["sentence_count"].sum() + 1
    
    cond_df = candidate_df.query("disease_id == @disease & gene_id != @gene")
    c = cond_df["sentence_count"].sum() + 1
    
    cond_df = candidate_df.query("disease_id != @disease & gene_id != @gene")
    d = cond_df["sentence_count"].sum() + 1
    
    c_table = np.array([[a, b], [c, d]])
    
    # Gather confidence interval
    delta, ci, corrected_ci = diffprop(c_table)
    lower_ci.append(ci[0])
    
    # Gather corrected odds ratio and p_values
    odds_ratio, p_value = fisher_exact(c_table, alternative='greater')
    odds.append(odds_ratio)
    p_val.append(p_value + epsilon)
    
    total_disease = candidate_df[candidate_df["disease_id"] == disease]["sentence_count"].sum()
    total_gene = candidate_df[candidate_df["gene_id"] == gene]["sentence_count"].sum()
    expected.append((total_gene * total_disease)/float(total))

candidate_df["nlog10_p_value"] = (-1 * np.log10(p_val))
candidate_df["co_odds_ratio"] = odds
candidate_df["co_expected_sen_count"] = expected
candidate_df["delta_lower_ci"] = lower_ci

In [ ]:
candidate_df.sort_values("nlog10_p_value", ascending=False).head(1000)

# Combine Sentence Marginal Probabilities

In this section we incorporate the marginal probabilites that are calculated from the bi-directional LSTM used in the [previous notebook](4.sentence-level-prediction.ipynb). For each sentence we grouped them by their disease-gene mention and report their marginal probabilites in different quantiles (0, 0.2, 0.4, 0.6, 0.8). Lastly we took the average of each sentence marginal to generate the "avg_marginal" column.

In [13]:
candidate_df = pd.read_csv("data/disease_gene_summary_stats.csv")
candidate_df = candidate_df[[column for column in candidate_df.columns if "lstm" not in column]]
candidate_df = candidate_df.drop(["disease_name", "gene_name"], axis=1)
candidate_df.head(2)

,disease_id,gene_id,nlog10_p_value,co_odds_ratio,co_expected_sen_count,delta_lower_ci,hetnet_labels
0,DOID:12849,3569,2.072881e-04,0.617773,62.795531,-0.001252,1
1,DOID:3070,567,3.358850e-09,0.192395,86.200979,-0.025416,-1


In [6]:
train_marginals_df = pd.read_table("data/training_set_marginals.tsv")
train_marginals_df.head(2)

,disease_id,gene_id,marginals
0,DOID:14330,4129,0.499679
1,DOID:1612,5111,0.134695


In [7]:
dev_marginals_df = pd.read_table("data/dev_set_marginals.tsv")
dev_marginals_df.head(2)

,disease_id,gene_id,marginals
0,DOID:10283,6732,0.062301
1,DOID:3070,3589,0.136521


In [8]:
candidate_marginals_df=(train_marginals_df
 .append(dev_marginals_df)
 .groupby(["disease_id", "gene_id"], as_index=False).mean()
)
candidate_marginals_df.head(2)

,disease_id,gene_id,marginals
0,DOID:0050156,21,0.385887
1,DOID:0050156,37,0.289145


In [9]:
dg_map = pd.read_csv("data/disease-gene-pairs-association.csv.xz")
dg_map.head(3)

/home/danich1/anaconda2/envs/snorkeling/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,entrez_gene_id,gene_symbol,doid_id,doid_name,sources,hetionet,n_sentences,has_sentence,partition_rank,split
0,1,A1BG,DOID:2531,hematologic cancer,NaN,0,8,1,0.85860,1
1,1,A1BG,DOID:1319,brain cancer,NaN,0,0,0,0.36785,0
2,1,A1BG,DOID:1324,lung cancer,NaN,0,2,1,0.15465,0


In [10]:
prior_df = pd.read_csv("data/observation-prior.csv")
prior_df['logit_prior_perm'] = prior_df.prior_perm.apply(logit)
prior_df.head(2)

,disease_id,gene_id,disease_associates,gene_associates,prior_perm,prior_perm_stderr,logit_prior_perm
0,DOID:2531,7124,550,47,0.870898,0.000558,1.908922
1,DOID:2531,7157,550,46,0.867094,0.000565,1.875505


In [14]:
candidate_df = (candidate_df
 .merge(candidate_marginals_df)
 .merge(dg_map[["doid_id", "entrez_gene_id","split"]], 
        left_on=["disease_id", "gene_id"], 
        right_on=["doid_id", "entrez_gene_id"]
       )
 .drop(["doid_id", "entrez_gene_id"], axis='columns')
 .merge(prior_df[["disease_id", "gene_id", "logit_prior_perm"]], on=["disease_id", "gene_id"])
)
candidate_df.head(2)

,disease_id,gene_id,nlog10_p_value,co_odds_ratio,co_expected_sen_count,delta_lower_ci,hetnet_labels,marginals,split,logit_prior_perm
0,DOID:12849,3569,0.000207,0.617773,62.795531,-0.001252,1,0.340116,0,0.644292
1,DOID:1324,6647,-0.000000,0.142479,194.551269,-0.044854,-1,0.192470,0,-1.379122


## Save the data to a file

In [15]:
candidate_df.to_csv("data/disease_gene_association_features.tsv", sep="\t", index=False)